In [ ]:
import gradio as gr
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib
from scipy.stats import kurtosis, skew
import traceback
from google.colab import drive

drive.mount('/content/drive')

# Load the trained models and preprocessing tools
try:
    hash_model = load_model('/content/drive/My Drive/hash_model/hashing_algorithm_cnn_model.h5')
    hash_scaler = joblib.load('/content/drive/My Drive/hash_model/scaler.pkl')
    hash_label_encoder = joblib.load('/content/drive/My Drive/hash_model/label_encoder.pkl')

    stream_model = load_model('/content/drive/My Drive/stream_model/stream_algorithm_cnn_model.h5')
    stream_scaler = joblib.load('/content/drive/My Drive/stream_model/scaler.pkl')
    stream_label_encoder = joblib.load('/content/drive/My Drive/stream_model/label_encoder.pkl')

    block_model = load_model('/content/drive/My Drive/block_model/block_algorithm_cnn_model.h5')
    block_scaler = joblib.load('/content/drive/My Drive/block_model/scaler.pkl')
    block_label_encoder = joblib.load('/content/drive/My Drive/block_model/label_encoder.pkl')
except Exception as e:
    print("Error loading models or preprocessing tools.")
    print(e)
    traceback.print_exc()

# Function to extract features and detect weaknesses from the input text
def extract_features(cipher_text):
    try:
        cipher_bytes = bytes.fromhex(cipher_text)
        cipher_array = np.frombuffer(cipher_bytes, dtype=np.uint8)

        mean = np.mean(cipher_array)
        std_dev = np.std(cipher_array)

        # Histogram for entropy calculation
        histogram, _ = np.histogram(cipher_array, bins=256, range=(0, 256))
        histogram = histogram / np.sum(histogram)  # Normalize histogram
        entropy = -np.sum(histogram * np.log2(histogram + 1e-10))

        kurt = kurtosis(cipher_array)
        skewness = skew(cipher_array)

        min_value = np.min(cipher_array)
        max_value = np.max(cipher_array)
        range_value = max_value - min_value
        unique_byte_count = len(np.unique(cipher_array))

        # Combine features into a single list
        features = [
            mean, std_dev, entropy, kurt, skewness, min_value, max_value, range_value, unique_byte_count
        ]

        # Define ideal ranges for each feature
        ideal_entropy = "7.5 to 8.0"  # Ideal entropy range for secure ciphertext
        ideal_skewness = "close to 0"  # Ideal skewness should be near zero for randomness
        ideal_kurtosis = "around 3.0"  # Ideal kurtosis for a normal distribution
        ideal_unique_byte_count = "above 200"  # Ideal count for a diverse byte range

        # Detect weaknesses with detailed explanations
        weakness_messages = []

        if entropy < 7.5:
            weakness_messages.append(
                f"Low entropy detected ({entropy:.2f}). Ideal range: {ideal_entropy}. "
                "This indicates insufficient randomness, making the ciphertext potentially susceptible to cryptanalysis."
            )
        elif entropy > 8.0:
            weakness_messages.append(
                f"High entropy detected ({entropy:.2f}). Ideal range: {ideal_entropy}. "
                "This could indicate excessive noise or incorrect encryption."
            )

        if kurt < 2.0 or kurt > 4.0:
            weakness_messages.append(
                f"Unusual kurtosis detected ({kurt:.2f}). Ideal value: {ideal_kurtosis}. "
                "This suggests that the byte distribution may be irregular, which can indicate poor randomness."
            )

        if abs(skewness) > 0.5:
            weakness_messages.append(
                f"High skewness detected ({skewness:.2f}). Ideal range: {ideal_skewness}. "
                "A skewed distribution indicates that certain byte values occur more frequently, reducing the ciphertext's randomness."
            )

        if unique_byte_count < 200:
            weakness_messages.append(
                f"Low unique byte count detected ({unique_byte_count}). Ideal value: {ideal_unique_byte_count}. "
                "A low number of unique byte values suggests that the encryption algorithm might not be using the full byte range effectively, leading to patterns."
            )

        return features, weakness_messages

    except Exception as e:
        print("Error extracting features from input.")
        print(e)
        traceback.print_exc()
        return None, ["Error extracting features from input."]

# Function to preprocess and predict using a specified model
def predict(model, scaler, label_encoder, cipher_text):
    features, weaknesses = extract_features(cipher_text)
    if features is None:
        return None, "Error extracting features from input text.", weaknesses

    try:
        features = np.array(features).reshape(1, -1)  # Reshape for scaling
        features = scaler.transform(features)  # Scale features
        features = features.reshape(1, 9, 1, 1)  # Reshape for CNN (assuming single-channel images)

        # Predict using the model
        prediction = model.predict(features)
        confidence = prediction.max()
        predicted_label_index = np.argmax(prediction, axis=1)
        predicted_label = label_encoder.inverse_transform(predicted_label_index)[0]

        return confidence, predicted_label, weaknesses
    except Exception as e:
        print("Error in prediction process.")
        print(e)
        traceback.print_exc()
        return None, "Prediction failed.", weaknesses

# Define the function that integrates with Gradio
def predict_algorithm(cipher_text):
    hash_confidence, hash_prediction, hash_weaknesses = predict(hash_model, hash_scaler, hash_label_encoder, cipher_text)
    stream_confidence, stream_prediction, stream_weaknesses = predict(stream_model, stream_scaler, stream_label_encoder, cipher_text)
    block_confidence, block_prediction, block_weaknesses = predict(block_model, block_scaler, block_label_encoder, cipher_text)

    # Find the prediction with the highest confidence score
    confidences = [hash_confidence, stream_confidence, block_confidence]
    predictions = [hash_prediction, stream_prediction, block_prediction]

    if None in confidences:
        return "An error occurred during prediction."

    # Identify the prediction with the highest confidence score
    max_index = np.argmax(confidences)
    prediction = predictions[max_index]

    # Combine weaknesses from all models
    all_weaknesses = set(hash_weaknesses + stream_weaknesses + block_weaknesses)
    weakness_output = "\n".join(all_weaknesses) if all_weaknesses else "No significant weaknesses detected."

    # Create an informative output string, listing all confidence values
    result = (
        f"Predicted Algorithm: {prediction}\n\n"
        f"Confidence Scores:\n"
        f" - Hashing Model: {hash_confidence:.4f} ({hash_prediction})\n"
        f" - Stream Cipher Model: {stream_confidence:.4f} ({stream_prediction})\n"
        f" - Block Cipher Model: {block_confidence:.4f} ({block_prediction})\n\n"
        f"Weakness Detection:\n{weakness_output}"
    )

    return result

# Update the Gradio interface
iface = gr.Interface(
    fn=predict_algorithm,
    inputs=gr.Textbox(label="Enter Cipher Text (Hexadecimal)"),
    outputs=gr.Textbox(label="Predicted Algorithm, Confidence Scores, and Weakness Detection"),
    title="CipherXPose: Cryptographic Algorithm and Weakness Detection",
    description="This tool predicts which cryptographic algorithm is most likely used for the provided cipher text and detects potential weaknesses in the encryption."
)

iface.launch()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 802ms/step
Predicted Algorithm (Majority Vote): RC4

Model Predictions:
 - Stream Cipher Model: RC4 (Confidence: 0.9995)
 - Hash Model: SHA256 (Confidence: 0.9995)
 - Block Cipher Model: 3DES (Confidence: 0.4679)
